# Schematics Cheat Sheet

---
## Types

### BaseType

`BaseType` come with the following parameters:

- `choices`: A list of choices
- `default`: default value, if not specified
- `deserialize_from`
- `export_level`
- `messages`
- `metadata`
- `required`: Default is False
- `serialize_when_none`: Default is True
- `serialized_name`: If serialize name is different from field name
- `validators`: List of `validate(value)` functions

### Simple Types

- `StringType(regex, max_length, min_length)`
- `BooleanType`
- `NumberType(min_value, max_value, strict)` -> `IntType`, `LongType`, `FloatType`, `DecimalType`

### Date and Time Types

- `DateType(formats)`
- `DateTimeType(formats, serialized_format, parser, tzd, convert_tz, drop_tzinfo)`
- `UTDateTimeType`
- `TimeStampType`
- `TimeDeltaType`
- `fixed_timezone`

### Nested Types

- `ModelType(model_spec)`
- `PolyModelType`
- `ListType(min_size, max_size`
- `DictType`
- `CompoundType`, `MultiType`

### Networking and Internet Types

- `IPAddressType(regex, max_length, min_length)`
- `IPv4Type(regex, max_length, min_length)`
- `IPv6Type(regex, max_length, min_length)`
- `MACAddressType(regex, max_length, min_length)`
- `URLType(fqdn=True, verify_exists=False)`
- `EmailType(regex, max_length, min_length)`


---
## Validation

We can validate the models using one or more of:

- Regex
- Validators (free function)
- Method named `validate_<field-name>`
- More info, see [validation.ipynb](validation.ipynb)

### Validate Using Regex

In [1]:
import schematics

class UnixUser(schematics.Model):
    name = schematics.types.StringType(regex=r"[a-z][a-z0-9.-_]+", required=True)

### Validate Using Validators (Free Functions)

In [2]:
def validate_phone_number(value):
    """Raise schematics.exceptions.ValidationError if failed."""
    
class Phone(schematics.Model):
    mobile = schematics.types.StringType(validators=[validate_phone_number])

### Validate a Field Using `validate_<field-name>`

In [3]:
class UnixUser(schematics.Model):
    alias = schematics.types.StringType(required=True)
    is_admin = schematics.types.BooleanType(default=False)
    
    def validate_is_admin(self, json_obj, is_admin_value):
        """Validate the field `is_admin`."""
        if json_obj["alias"] == "root" and is_admin_value is not True:
            raise schematics.exceptions.ValidationError("root must be an admin.")

### Validate Using Custom Type

In [4]:
class SimpleUSPhoneType(schematics.types.StringType):
    def validate_first_digit(self, value):
        pass
            
    def validate_format(self, value):
        pass
    
    # Write more validate_* if needed

class Contact2(schematics.Model):
    name = schematics.types.StringType(required=True)
    mobile_phone = SimpleUSPhoneType()
    work_phone = SimpleUSPhoneType()

---
## Nested Structures

Reference: [nested.ipynb](nested.ipynb)

In [5]:
class UnixUser(schematics.Model):
    alias = schematics.types.StringType(required=True)
    is_admin = schematics.types.BooleanType(default=False)
    
class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(choices=["mobile", "work", "home", "other"])
    
class Contact(schematics.Model):
    name = schematics.types.StringType(required=True)
    
    # Nested: A single object
    unix_user = schematics.types.ModelType(UnixUser, serialized_name="unixUser")
    
    # Nested: Many objects
    phones = schematics.types.ListType(schematics.types.ModelType(Phone))

---
## Other Topics

* [Date and Time Fields](date_time.ipynb)
* [Exploring a Model](exploring_a_model.ipynb)
* [Generate random models](random_models.ipynb)
* [Rogue Fields](rogue_fields.ipynb)
* [Validate Against Expected Data](expected.ipynb)
